#### _______IMPORTS Y AUTENTICACIONES_______

In [3]:
# IMPORTS
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# CREDENCIALES
CLIENT_ID = '768f68e91bc641f89df701439c4b0255'          # Variar en función del rate limit
CLIENT_SECRET = '671c7e27d76241cca8082b751071789a'      # Variar en función del rate limit

# AUTENTICACIÓN
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=20)

"""
AQUÍ PONEMOS LAS CLAVES DE TODAS PARA IR VARIANDO EN FUNCIÓN DE CUANDO SALGAN RATE LIMITS
CLAVES IRINA
768f68e91bc641f89df701439c4b0255
671c7e27d76241cca8082b751071789a

CLAVES ELENA
fec1ea7ca457405e81c273b6d7b1dbf7
2811d856f33d4404920796597621b514

CLAVES GEMA
498ba98d2f134171840b8e8a508522a9
7fdc4fc6f9a84f368a60b4bec19aefb7

CLAVES MARIA
74da384a9fab4748ab050fb0b532f11c
9f250736046943758dd217b8c795f300
"""

'\nAQUÍ PONEMOS LAS CLAVES DE TODAS PARA IR VARIANDO EN FUNCIÓN DE CUANDO SALGAN RATE LIMITS\nCLAVES IRINA\n768f68e91bc641f89df701439c4b0255\n671c7e27d76241cca8082b751071789a\n\nCLAVES ELENA\nfec1ea7ca457405e81c273b6d7b1dbf7\n2811d856f33d4404920796597621b514\n\nCLAVES GEMA\n498ba98d2f134171840b8e8a508522a9\n7fdc4fc6f9a84f368a60b4bec19aefb7\n\nCLAVES MARIA\n74da384a9fab4748ab050fb0b532f11c\n9f250736046943758dd217b8c795f300\n'

# ________CONSULTA DE 250 ÁLBUMES POR GÉNERO________

In [4]:
albums_per_genre = 250   # Número máximo de albumes que queremos obtener por género
genres = ["rock", "R&B", "indie", "pop"]   # Lista de géneros que queremos consultar
datas_albums = {}        # Diccionario para guardar todos los albumes organizados por género

# Iteramos cada género de la lista
for genre in genres:
    print(f"Obteniendo {albums_per_genre} álbumes del género {genre}...") # Print de control
    albums_info = []   # Lista para almacenar la información de los álbumes de este género
    offset = 0         # Nos ayuda a controlar la paginación 

    # Mientras no tengamos suficientes albumes y no superemos 1000 resultados...:
    while len(albums_info) < albums_per_genre and offset < 1000:
        # 1. Busca artistas en Spotify del género actual
        artists = sp.search(
            q=f"genre:{genre}",   # Filtramos por género
            type="artist",        # Queremos resultados de tipo artista
            limit=50,             # El máximo que nos permiten es 50 artistas por consulta
            offset=offset,        # Offset para paginar los resultados
            market="ES"           # Porque queremos extraer información del mercado español, pero es opcional. Si no, sacaría a nivel global.
        )["artists"]["items"]     # Extraemos la lista de artistas
        offset += 50              # Avanzamos la paginación para la siguiente búsqueda

        # 2. Para cada artista encontrado, buscamos sus albumes
        for artist in artists:
            artist_albums = sp.artist_albums(
                artist["id"],        # Saxamos el ID único del artista
                album_type="album",  # Indicamos que solo queremos albumes completos (no singles, etc.)
                limit=50             # Igual, máximo 50 albumes por consulta
            )["items"]

            # Recorremos cada album encontrado
            for album in artist_albums:
                # Sacamos el año de lanzamiento (los 4 primeros caracteres de release_date)
                year = int(album["release_date"][:4])
                # Y solo consideramos los albumes publicados entre 2010 y 2015
                if 2010 <= year <= 2015:
                    # Entonces hacemos otra llamada para pedir del album para sacar toda la información que nos interesa usando su id
                    album_details = sp.album(album["id"]) 

                    # Creamos un diccionario con toda la información del album
                    info = {}
                    info["genre"] = genre                        # Género del album
                    info["album_name"] = album["name"]           # Nombre del album
                    info["album_id"] = album["id"]               # ID del album en Spotify
                    info["release_date"] = album["release_date"] # Fecha de lanzamiento
                    info["total_tracks"] = album["total_tracks"] # Número de canciones del album
                    info["popularity"] = album_details["popularity"]  # Popularidad del album (0-100)

                    # Para cada album, reiniciamos la lista de artistas
                    artists_album = []
                    for a in album["artists"]:
                        artists_album.append(a["name"])
                    # Guardamos la cadena de artistas solo una vez
                    info["artists_album"] = ", ".join(artists_album)

                    # Haremos otra llamada para obtener algo más de información sobre el artista
                    main_artist = album["artists"][0]                  # Tomamos el primer artista (principal)
                    artist_details = sp.artist(main_artist["id"])      # Obtenemos sus detalles
                    info["artist_followers"] = artist_details["followers"]["total"]  # Número de seguidores en Spotify
                    info["artist_popularity"] = artist_details["popularity"]         # Popularidad del artista
                    info["artist_genres"] = ", ".join(artist_details["genres"])      # Otros géneros asociados al artista

                    # Añadimos el diccionario a la lista de albumes del género
                    albums_info.append(info)

                # Si ya tenemos suficientes albumes, salimos del bucle
                if len(albums_info) >= albums_per_genre:
                    break

            # También rompemos este bucle si ya tenemos suficientes
            if len(albums_info) >= albums_per_genre:
                break

    # Guardamos los álbumes del género actual en el diccionario principal
    datas_albums[genre] = albums_info

    # Imprimimos la cantidad de albumes obtenidos para este género
    print(f"✔ {len(albums_info)} álbumes obtenidos de {genre}")
    print('=================================')


Obteniendo 250 álbumes del género rock...
✔ 250 álbumes obtenidos de rock
Obteniendo 250 álbumes del género R&B...
✔ 250 álbumes obtenidos de R&B
Obteniendo 250 álbumes del género indie...
✔ 250 álbumes obtenidos de indie
Obteniendo 250 álbumes del género pop...
✔ 250 álbumes obtenidos de pop


### _________CONVERTIR JSON EN CSV_________

In [ ]:
# 1. Creamos una lista vacía para guardar todos los álbumes de todos los géneros
all_albums = []

# 2. Recorremos el diccionario grande datas_albums
for genre, albums in datas_albums.items():
    # albums es la lista de diccionarios con info de cada álbum
    for album in albums:
        all_albums.append(album)   # Vamos añadiendo cada álbum a la lista total

# 3. Convertimos la lista completa a un DataFrame de pandas
df_albumes = pd.DataFrame(all_albums)

# 4. Guardamos el DataFrame en un archivo CSV
df_albumes.to_csv('albums_v2.csv', index=False)

print("✔ Archivo CSV creado con éxito: albums_v2.csv")

✔ Archivo CSV creado con éxito: albums_v2.csv


# ________MÉTODO PARA SACAR ARTISTAS X GÉNERO________
Lo usaremos después en LastFM

In [6]:
# Extracción de todos los artistas únicos a través de los álbumes para CSV
all_artists = []
for genre_albums in datas_albums.values():
    for info in genre_albums:
        nombres = info["artists_album"].split(", ")
        for i in nombres:
            all_artists.append(i.strip())

setartist = set(all_artists)
df_artistas = pd.DataFrame(list(setartist), columns=["artist"])
df_artistas.to_csv('artistas_v2.csv', index=False)

In [7]:
setartist = set(all_artists)
df_artistas = pd.DataFrame(list(setartist), columns=["artist"])
df_artistas.to_csv('artistas_v2.csv', index=False)

print(f"✔ Se han extraído {len(df_artistas)} artistas únicos")

✔ Se han extraído 333 artistas únicos


# ________CONSULTA DE 250 CANCIONES POR GÉNERO________

In [ ]:
genres = ["rock", "R&B", "indie", "pop"] # Lista de géneros que queremos consultar
songs_per_genre = 250                    # Número máximo de canciones que queremos obtener por género

# Lista que guardará todas las canciones de todos los géneros
all_songs_list = []

# Iteramos por cada género de la lista
for genre in genres:
    print(f"Obteniendo {songs_per_genre} canciones del género {genre}...")
    
    offsets = range(0, songs_per_genre, 50) # Inicia en 0, termina en 150, y va haciendo saltos 50 en 50, que es lo que nos permite Spotify
    
    for offset in offsets:
        results = sp.search(
            q=f"genre:{genre} year:2010-2015", # Búsqueda de canciones del género y en ese rango de años
            type="track",                      # Queremos resultados del tipo "track" (canción)
            limit=50,                          # Número máximo por petición, lo que nos deja Spotify
            offset=offset,                     # Para paginar
            market="ES"                        # Como hemos hecho con albums, en el mercado español
        )
        # Iteramos cada canción dentro de los resultados 
        for track in results["tracks"]["items"]:
            # Y construimos una lista de artistas
            artists_track = []
            for a in track["artists"]:
                artists_track.append(a["name"]) # Añadimos el nombre del artista en esa lista

            # Creamos un diccionario con toda la información que nos interesa de cada canción
            info = {
                "genre": genre,                                     # Género de la canción
                "track_name": track["name"],                        # Nombre del artista
                "track_id": track["id"],                            # ID de la canción
                "artists_track": ", ".join(artists_track),          # Lista de artistas convertida en string
                "album_name": track["album"]["name"],               # Nombre del álbum de la canción
                "release_date": track["album"]["release_date"],     # Fecha de lanzamiento del album al que pertenece la canción
                "popularity": track["popularity"],                  # Popularidad
                "duration_ms": track["duration_ms"]                 # Duración
            }
            # Añade la información a la lista principal
            all_songs_list.append(info)

print("===============")
print(f"✔ Se han obtenido un total de {len(all_songs_list)} canciones de todos los géneros.")

# Creamos un DataFrame y su CSV.
df_canciones = pd.DataFrame(all_songs_list)
df_canciones.to_csv("canciones_v2.csv", index=False)


Obteniendo 250 canciones del género rock...
Obteniendo 250 canciones del género R&B...
Obteniendo 250 canciones del género indie...
Obteniendo 250 canciones del género pop...
✔ Se han obtenido un total de 1000 canciones de todos los géneros.
